In [1]:
import cv2
import os
import numpy as np
from collections import Counter

In [2]:
def histogram_equalization(image):

    # 计算直方图
    hist, bins = np.histogram(image.flatten(), 256, [0, 256])

    # 计算累积直方图
    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()

    # 创建映射表
    cdf_m = np.ma.masked_equal(cdf, 0)
    cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
    cdf = np.ma.filled(cdf_m, 0).astype('uint8')

    # 应用映射表
    equalized_image = cdf[image]

    return equalized_image

# 导入训练向量

In [3]:
# 返回的是480张100*100的矩阵，和index
listdir = os.listdir('./face_train')
names = [d for d in listdir if not d.startswith('.')]
#print(names)
images = []
target = []
for index,dir in enumerate(names):
    for i in range(1, 11):
        gray = cv2.imread('./face_train/%s/%d.jpg' % (dir, i))  # 三维图片
        
        #图片均衡化
        gray_equalized = histogram_equalization(gray)
        
        gray_ = gray_equalized[:, :, 0]  # 二维数组
        gray_ = cv2.resize(gray_, dsize=(100, 100))      
        images.append(gray_)
        target.append(index)
images = np.asarray(images)
target = np.asarray(target)

In [4]:
# 转换成矩阵
#图像数据转换特征矩阵
image_data = []

for image in images:
    #转换成一维的数组
    data = image.flatten()
    image_data.append(data)

In [5]:
#转换为numpy数组
X = np.array(image_data)
print(type(X))
print(X.shape)
print(X)

<class 'numpy.ndarray'>
(200, 10000)
[[243 250 245 ... 186 158 199]
 [244 250 248 ... 213 213 213]
 [243 243 243 ... 195 196 204]
 ...
 [212 212 212 ...  79 193 201]
 [191 191 191 ...  34  74 127]
 [211 211 211 ...  14  17  22]]


# 将训练数据进行PCA

In [6]:
# 对数据进行中心化，即每一列减去该列的均值
X_train_centered = X - np.mean(X, axis=0)

In [7]:
# 主成分数量
k = 100
# 对训练矩阵进行SVD
U, S, Vt = np.linalg.svd(X_train_centered, full_matrices=False)
# 选择前100个主成分
X_train_pca = U[:, :k]
print(X_train_pca.shape)
# 构建变换矩阵P
P = np.dot(X.T, X_train_pca)
print(P)

(200, 100)
[[-2.41937190e+02 -8.33315987e+01 -2.05164004e+02 ... -3.64924770e+00
   3.34063307e+00 -2.46692790e+00]
 [-3.46914288e+02 -1.61265508e+02 -2.36464779e+02 ...  1.30922526e+01
  -1.11345452e+01 -1.88221905e+01]
 [-4.61506429e+02 -2.24270100e+02 -2.13278374e+02 ... -5.42448327e+00
   7.42002786e-01 -1.73155215e+01]
 ...
 [-8.27276244e+02 -3.64422213e+02 -2.39993616e+02 ... -5.09180406e+00
  -1.43875951e+01  4.08727643e+01]
 [-7.69798771e+02 -4.15184129e+02 -2.20611336e+02 ...  1.14341731e+01
  -3.23972705e+01  5.73801333e+01]
 [-7.33773325e+02 -4.07356797e+02 -2.66366008e+02 ...  1.70240417e+01
  -3.47736668e+01 -2.40345741e+01]]


In [8]:
cap = cv2.VideoCapture(0)
# 人脸检测
face_detector = cv2.CascadeClassifier('E:/tools/opencv/sources/data/haarcascades/haarcascade_frontalface_alt2.xml')
while True:
    flag,frame = cap.read()
    if not flag:
        break
   
    gray = cv2.cvtColor(frame,code = cv2.COLOR_BGR2GRAY)
    
    # 提取测试向量
    faces = face_detector.detectMultiScale(gray, minNeighbors=12)
    for x,y,w,h in faces:       
        
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face,dsize=(100,100))
        
        face_equalized = histogram_equalization(face)
        
        imgs = []
        imgs.append(face_equalized)
        imgs = np.asarray(imgs)
        
        #图像数据转换特征矩阵
        image_data = []

        data = imgs.flatten()
        image_data.append(data)
        
        #转换为numpy数组
        test_vector = np.array(image_data)
        #print(test_vector)

        # 归一化测试向量
        test_vector_centered = test_vector - np.mean(test_vector)
        #print(test_vector_centered)
        
        # 对测试向量PCA
        test_vector_pca = np.dot(test_vector_centered,P)
        
        #print(test_vector_pca)

        # 初始化一个列表来存储每个训练向量与当前测试向量的欧几里得范数
        distances = []

        for j in range(len(X_train_pca)):
            train_vector = X_train_pca[j]

            dist = np.sqrt(np.sum((train_vector-test_vector_pca) ** 2))
            distances.append(dist)

        # 找到最近的k个邻居
        nearest_indices = np.argsort(distances)[:3]
        print(nearest_indices)
        # 找到最频繁的索引
        counts = Counter(nearest_indices//10)
        most_common_index = counts.most_common(1)[0][0]
        
        print(most_common_index)

        # 人脸辨识，返回index和置信度
        y_ = most_common_index
        
        label = names[y_]
        
        print('这个人是：%s。'%(label))
        cv2.rectangle(frame,pt1 = (x,y),pt2 = (x+w,y+h),color=[0,0,255], thickness = 2)
        cv2.putText(frame,text = label, org = (x, y-10),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1.5,
                    color=[0,0,255],
                    thickness=2)
    cv2.imshow('face',frame)
    key = cv2.waitKey(1000//30)
    
    if key == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()


[10 19 16]
1
这个人是：Chen Jiayin。
[10 16 11]
1
这个人是：Chen Jiayin。
[13 10 16]
1
这个人是：Chen Jiayin。
[13 14 16]
1
这个人是：Chen Jiayin。
[13 16 15]
1
这个人是：Chen Jiayin。
[19 10 18]
1
这个人是：Chen Jiayin。
[15 16 13]
1
这个人是：Chen Jiayin。
[15 16 18]
1
这个人是：Chen Jiayin。
[18 19 15]
1
这个人是：Chen Jiayin。
[14 13 16]
1
这个人是：Chen Jiayin。
[15 18 16]
1
这个人是：Chen Jiayin。
[13 15 18]
1
这个人是：Chen Jiayin。
[77 78 76]
7
这个人是：Li Xiufang。
[13 14 16]
1
这个人是：Chen Jiayin。
[19 10 18]
1
这个人是：Chen Jiayin。
[13 10 16]
1
这个人是：Chen Jiayin。
[15 16 18]
1
这个人是：Chen Jiayin。
[10 19 12]
1
这个人是：Chen Jiayin。
[10 19 12]
1
这个人是：Chen Jiayin。
[13 15 16]
1
这个人是：Chen Jiayin。
[19 11 16]
1
这个人是：Chen Jiayin。
[19 18 15]
1
这个人是：Chen Jiayin。
[18 19 15]
1
这个人是：Chen Jiayin。
[19 10 18]
1
这个人是：Chen Jiayin。
[18 15 19]
1
这个人是：Chen Jiayin。
[11 19 16]
1
这个人是：Chen Jiayin。
[11 19 10]
1
这个人是：Chen Jiayin。
[76 77 75]
7
这个人是：Li Xiufang。
[75 77 76]
7
这个人是：Li Xiufang。
[76 75 77]
7
这个人是：Li Xiufang。
[76 77 75]
7
这个人是：Li Xiufang。
[151  76 153]
15
这个人是：Yu Hong。
[75 76 74]
7


[15 18 16]
1
这个人是：Chen Jiayin。
[10 19 11]
1
这个人是：Chen Jiayin。
[10 11 19]
1
这个人是：Chen Jiayin。
[10 13 11]
1
这个人是：Chen Jiayin。
[10 13 11]
1
这个人是：Chen Jiayin。
[10 19 13]
1
这个人是：Chen Jiayin。
[10 19 13]
1
这个人是：Chen Jiayin。
[10 11 19]
1
这个人是：Chen Jiayin。
[10 11 13]
1
这个人是：Chen Jiayin。
[10 11 19]
1
这个人是：Chen Jiayin。
[10 13 11]
1
这个人是：Chen Jiayin。
[10 19 11]
1
这个人是：Chen Jiayin。
[10 19 11]
1
这个人是：Chen Jiayin。
[10 19 12]
1
这个人是：Chen Jiayin。
[10 19 12]
1
这个人是：Chen Jiayin。
[10 19 18]
1
这个人是：Chen Jiayin。
[10 19 18]
1
这个人是：Chen Jiayin。
[15 18 19]
1
这个人是：Chen Jiayin。
[10 19 11]
1
这个人是：Chen Jiayin。
[10 19 12]
1
这个人是：Chen Jiayin。
[13 10 16]
1
这个人是：Chen Jiayin。
[10 19 11]
1
这个人是：Chen Jiayin。
[10 11 19]
1
这个人是：Chen Jiayin。
[11 16 19]
1
这个人是：Chen Jiayin。
